In [7]:
from EventDriven.backtest import OptionSignalBacktest
import pandas as pd
import numpy as np
from IPython.display import clear_output
from copy import deepcopy

In [8]:
import json
keys = [8, 9, 10, 11, 12 ]
imports_trades = {key: {'trades': pd.read_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_trades_{key}.csv').iloc[:, 1:]} for key in keys}

for key in keys:
    with open(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/input/profitable_weights_{key}.json', 'r') as f:
        weights = json.load(f)
        symbol_list = imports_trades[key]['trades'].Ticker.unique()
        untraded_symbols = [s for s in weights.keys() if s not in imports_trades[key]['trades'].Ticker.unique()]
        for s in untraded_symbols:
            weights.pop(s)
        imports_trades[key]['weights'] = weights

In [9]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
initial_capital = 20_000
capital_tracking = {}
objects = {}
for key in keys:
  print(f"Starting backtest for key: {key}, with initial capital: {initial_capital}")
  ## Set the trades and weights
  trades_ = imports_trades[key]['trades']
  weights = imports_trades[key]['weights']
  objects[key] = {}
  print(weights)
  
  ## Produce max cash map
  max_cash = {}
  cash = 20_000
  for s, w in weights.items():
      if w * cash > 200:
          max_cash[s] = 2
      elif w * cash > 100:
          max_cash[s] = 1
      else:
          max_cash[s] = 0.5
  print(max_cash)

  ## Initialize the backtest class
  evb_backtest = OptionSignalBacktest(trades_, initial_capital=initial_capital)

  ## Set the attributes
  w_map = {x: w  * 0.85 for x, w in weights.items()} ## 85% of the weights for each stock
  evb_backtest.portfolio.weight_map = w_map
  evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 50
  evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
  evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
  evb_backtest.portfolio.order_settings = {'type': 'naked',
  'specifics': [{'direction': 'long',
    'rel_strike': .850,
    'dte': 300,
    'moneyness_width': 0.35},
    {'direction': 'short',
    'rel_strike': .60,
    'dte': 300,
    'moneyness_width': 0.35}
  ],
  'name': 'vertical_spread'}
  evb_backtest.portfolio.max_contract_price=max_cash
  evb_backtest.executor.commission_rate=0.65/100
  evb_backtest.executor.max_slippage_pct=0.075
  evb_backtest.portfolio.roll_map=3
  evb_backtest.portfolio.dte_reduction_factor=60
 
  custom_attr = {
      'max_cash': max_cash,
      'w_map': w_map,
      'initial_capital': initial_capital,
      'commission_rate': 0.65/100,
      'max_slippage_pct': 0.075,
      'roll_map': 3,
      'dte_reduction_factor': 60,
      'liquidity_threshold': 50,
      'lookback': 10,
      'data_availability_threshold': 0.5
  }
  ## Run the backtest
  evb_backtest.run()

  ## Get the ending capital
  ending_capital = evb_backtest.portfolio._equity['Total'][-1]
  initial_capital = ending_capital
  capital_tracking[key] = ending_capital
  objects[key]['WEIGHTS'] = w_map
  objects[key]['END_CASH'] = ending_capital
  objects[key]['TEST_RESULT'] = {}
  objects[key]['TEST_RESULT']['BACKTESTER'] = evb_backtest.portfolio
  objects[key]['TEST_RESULT']['AGG'] = evb_backtest.portfolio.aggregate()
  objects[key]['CUSTOM_ATTR'] = custom_attr

  clear_output(wait=True)

Starting backtest for key: 12, with initial capital: 41069.91377530724
{'TSLA': 0.12796432318191464, 'META': 0.11516488953289876, 'NFLX': 0.081963368341139, 'AAPL': 0.06739746345132278, 'AMZN': 0.04915178987041673, 'COST': 0.06935520353532351, 'BA': 0.028817628988948133, 'SBUX': 0.02479934071090783}
{'TSLA': 2, 'META': 2, 'NFLX': 2, 'AAPL': 2, 'AMZN': 2, 'COST': 2, 'BA': 2, 'SBUX': 2}
Processing event: MARKET
Processing event: SIGNAL
Not generating order because:MAX_PRICE_TOO_LOW at 2, adjusted to 2.4 SignalEvent type:LONG, symbol=NFLX, date:2025-01-03 00:00:00, Order Settings=None,Max Contract Price:None , signal_id:NFLX20250103LONG 
Processing event: SIGNAL
Buy Details
Position: {'long': ['AAPL20250919C260'], 'short': ['AAPL20250919C265'], 'trade_id': '&L:AAPL20250919C260&S:AAPL20250919C265', 'close': 2.0}, Date: 2025-01-03, Signal: SignalEvent type:LONG, symbol=AAPL, date:2025-01-03 00:00:00, Order Settings=None,Max Contract Price:None , signal_id:AAPL20250103LONG
Max Contract Price

In [10]:
ending_capital

40548.85167454567

In [11]:
objects[9]

{'WEIGHTS': {'NVDA': 0.21946994598983638,
  'TSLA': 0.12043574175666279,
  'AMD': 0.0664205211648468,
  'AAPL': 0.1073903358112171,
  'NFLX': 0.0678711494971221,
  'COST': 0.07324583721652256,
  'AMZN': 0.048585943103089,
  'BA': 0.054126317271090456,
  'SBUX': 0.05305851057239966},
 'END_CASH': 15588.932384904609,
 'TEST_RESULT': {'BACKTESTER': <EventDriven.portfolio.OptionSignalPortfolio at 0x1431dac90>,
  'AGG': Start                                                    2022-01-04 00:00:00
  End                                                      2022-12-30 00:00:00
  Duration                                                   360 days 00:00:00
  Exposure Time [%]                                                      73.75
  Equity Final [$]                                                    15588.93
  Equity Peak [$]                                                 26933.266208
  Return [%]                                                        -42.120156
  Buy & Hold Return [%]       

In [12]:
imports_trades[9]['trades'].PnL.sum(),imports_trades[9]['trades'].PnL.shape

(-12820.857005641003, (25,))

In [13]:
key = 9
objects[key]['TEST_RESULT']['BACKTESTER'].get_trades().PnL.sum(), imports_trades[key]['trades'].PnL.sum(), imports_trades[key]['trades'].PnL.shape, objects[key]['TEST_RESULT']['BACKTESTER'].get_trades().PnL.shape

AttributeError: 'OptionSignalPortfolio' object has no attribute 'get_trades'

In [62]:

imports_trades[9]['weights']['NVDA'] * 20000

3488.247915346469